In [1]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning_intro.ex4 import *

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

spotify = pd.read_csv('../data/raw/deep_neural_networks/spotify.csv')

X = spotify.copy().dropna()
y = X.pop('track_popularity')
artists = X['track_artist']

features_num = ['danceability', 'energy', 'key', 'loudness', 'mode',
                'speechiness', 'acousticness', 'instrumentalness',
                'liveness', 'valence', 'tempo', 'duration_ms']
features_cat = ['playlist_genre']

preprocessor = make_column_transformer(
    (StandardScaler(), features_num),
    (OneHotEncoder(), features_cat),
)

# We'll do a "grouped" split to keep all of an artist's songs in one
# split or the other. This is to help prevent signal leakage.
def group_split(X, y, group, train_size=0.75):
    splitter = GroupShuffleSplit(train_size=train_size)
    train, test = next(splitter.split(X, y, groups=group))
    return (X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test])

X_train, X_valid, y_train, y_valid = group_split(X, y, artists)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
y_train = y_train / 100 # popularity is on a scale 0-100, so this rescales to 0-1.
y_valid = y_valid / 100

input_shape = [X_train.shape[1]]
print("Input shape: {}".format(input_shape))

2025-05-18 21:14:56.416452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747595696.449106     879 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747595696.461578     879 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 21:14:56.590092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Input shape: [18]


In [3]:
model = keras.Sequential([
    layers.Dense(1, input_shape=input_shape),
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
    verbose=0, # suppress output since we'll plot the curves
)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1747595861.185309     879 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
W0000 00:00:1747595861.832407   27171 gpu_backend_lib.cc:579] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  ipykernel_launcher.runfiles/cuda_nvcc
  ipykern/cuda_nvcc
  
  /usr/local/cuda
  /home/caujoulat/

InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/home/caujoulat/miniforge3/envs/EnvDL/li

In [4]:
# Start the plot at epoch 10
history_df.loc[10:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

NameError: name 'history_df' is not defined

## 1) Evaluate baseline

In [5]:
# View the solution (Run this cell to receive credit!)
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

The gap between these curves is quite small and the validation loss never increases, so it's more likely that the network is underfitting than overfitting. It would be worth experimenting with more capacity to see if that's the case.

In [6]:
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=50,
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


E0000 00:00:1747597371.533574   27163 buffer_comparator.cc:157] Difference at 16: 0, expected 264.876
E0000 00:00:1747597371.533714   27163 buffer_comparator.cc:157] Difference at 17: 0, expected 253.655
E0000 00:00:1747597371.533725   27163 buffer_comparator.cc:157] Difference at 18: 0, expected 244.343
E0000 00:00:1747597371.533730   27163 buffer_comparator.cc:157] Difference at 19: 0, expected 250.485
E0000 00:00:1747597371.533733   27163 buffer_comparator.cc:157] Difference at 20: 0, expected 246.392
E0000 00:00:1747597371.533737   27163 buffer_comparator.cc:157] Difference at 21: 0, expected 249.136
E0000 00:00:1747597371.533740   27163 buffer_comparator.cc:157] Difference at 22: 0, expected 256.307
E0000 00:00:1747597371.533744   27163 buffer_comparator.cc:157] Difference at 23: 0, expected 246.965
E0000 00:00:1747597371.533747   27163 buffer_comparator.cc:157] Difference at 24: 0, expected 256.769
E0000 00:00:1747597371.533750   27163 buffer_comparator.cc:157] Difference at 25: 

InternalError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/home/caujoulat/miniforge3/envs/EnvDL/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/home/caujoulat/miniforge3/envs/EnvDL/li

## 2) Add capacity

In [7]:
# View the solution (Run this cell to receive credit!)
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

Now the validation loss begins to rise very early, while the training loss continues to decrease. This indicates that the network has begun to overfit. At this point, we would need to try something to prevent it, either by reducing the number of units or through a method like early stopping. (We'll see another in the next lesson!)

## 3) Define Early Stopping Callback

In [9]:
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

# YOUR CODE HERE: define an early stopping callback
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> The `min_delta` argument should be 0.001.